<a href="https://colab.research.google.com/github/HidekiAI/ML-manga109-OCR/blob/trunk/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# In CoLab

Before you begin, click on: Runtime -> Change runtime type -> select FREE runtime that supports GPU (i.e. T4)

Be careful on which to choose (i.e. TPU v2), some will crash with "Your session crashed after using all available RAM."

You should get output of something like this:

```text
Ultralytics YOLOv8.2.21 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, save_period=1, ... , device=0, ... , save_dir=runs/detect/train
...
Starting training for 50 epochs...

    Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
     1/50      4.12G      ...
```

Where it basically reports:

-   usage of `CUDA` on torch
-   `device=0` indicating that it's using single GPU
-   training indicates usage of GPU

As a comparison, it took about ~5 hours on a single CPU to complete 1 epoch, while it did about (almost finished) 3 epochs in 2.5 hours (before CoLab shut me down because it wante me to pay for the rest).  All in all, even with 1 GPU, it is a significant difference in time.

## Load minimal pyenv

Skip this if running on CoLab, only useful for local Jupyter-Notebook


In [ ]:
#!/bin/bash
%pip install google-colab

## Libs

-   Ultralitics YOLO


In [ ]:
#!/bin/bash
%pip install ultralytics
%pip install torch

For CoLab, you must first make sure remote drive is mounted. To align BASH and Python scripts to work on multiple platform, for local, you'd need to either soft-link (or junction) and/or mount (i.e. `mount bind`).

Note that below is ONLY necessary for Google CoLab to access your Google Drive. If on Notepad/Jupyter, do the following instead (not exact, just the example):

-   Linux: make sure to `ln -sv ~/Google/MyDrive /content/drive` to softlink your Google G-Drive as `/content/drive`
-   Windows: From DOS Command Prompt (right clock to launch as Admin) `mklink.exe /D "C:/content/drive" "C:/Users/HidekiAI/Google/MyDrive/"` to create a dir-junction


In [ ]:
#!/usr/bin/python
# No need to execute this if running locally, this is only for Google CoLab usage
import os
from google.colab import drive

drive.mount('/content/drive')
print(f"Current directory: {os.getcwd()}")

mount_drive = '/content/drive/MyDrive'
if not os.path.exists(mount_drive):
    print(f"Mount drive '{mount_drive}' does not exist")

dir_content = os.listdir(mount_drive)
print(f"Directory content: {dir_content}")

## Constants

Where are my data, where do I save my trained data and progress


In [ ]:
#!/usr/bin/python
import os

global mount_drive, root_src_data_path, root_dst_data_path, train_path, val_path, test_path, data_yaml_file_path, save_dir
mount_drive = '/content/drive/MyDrive'
root_src_data_path = mount_drive + \
    '/projects/ML-manga-ocr-rust/data/text_detection/'
root_dst_data_path = mount_drive + \
    '/projects/ML-manga-ocr-rust/data/text_detection/'
train_path = os.path.join(root_src_data_path, 'images/train')
val_path = os.path.join(root_src_data_path, 'images/val')
test_path = os.path.join(root_src_data_path, 'images/test')
data_yaml_file_path = os.path.join(root_dst_data_path, 'data.yaml')

if not os.path.exists(mount_drive):
    print(f"Mount drive {mount_drive} does not exist")

# NOTE: YOLO will set default value for 'save_dir=runs/detect/train' (if resume=False) and/or 'save_dir=YOLOv8/yolov8s' (if resume=True)
# engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, mixup=0.0, copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, tracker=botsort.yaml, save_dir=runs/detect/train
# print CURRENT directory:
# on CoLab, this will print '/content', as in '/content/YOLOv8/yolov8s/weights'
print(f"Current directory: {os.getcwd()}")
# if we're re-running this script, we're already in the destination directory, so go up a level
if os.getcwd().endswith("data"):
    print(f"Current directory ends with 'data', going up a level")
    os.chdir('..')
    print(f"New directory: {os.getcwd()}")

# ls /content/drive
dir_content = os.listdir(mount_drive)
print(f"Directory content: {dir_content}")

save_dir = os.path.join(os.getcwd(), 'runs/detect/train')
if not os.path.exists(save_dir):
    print(f"Save directory {save_dir} does not exist, creating...")
    os.makedirs(save_dir)

# if cwd starts with "C:\", then we are in Windows, so switch paths
if os.getcwd().startswith("C:\\") or os.getcwd().startswith("c:\\"):
    # NOTE: RELATIVE PATHS WILL NOT work if anywhere in the script, there is a call to `os.chdir()`!
    # if cwd is already at a dir where "runs" dir exists, we're still in the already-changed directory, so go up a level
    root_src_data_path = os.path.join(os.getcwd(), '../../data/text_detection')
    root_dst_data_path = os.path.join(os.getcwd(), './data/')
    train_path = os.path.join(root_src_data_path, 'images/train')
    val_path = os.path.join(root_src_data_path, 'images/val')
    test_path = os.path.join(root_src_data_path, 'images/test')
    data_yaml_file_path = os.path.join(root_dst_data_path, 'data.yaml')

# validate paths exist, if not panic
if not os.path.exists(root_src_data_path):
    print(f"Source data path {root_src_data_path} does not exist")
    raise SystemExit(1)
if not os.path.exists(root_dst_data_path):
    print(f"Destination data path {root_dst_data_path} does not exist")
    raise SystemExit(1)
if not os.path.exists(train_path):
    print(f"Train path {train_path} does not exist")
    raise SystemExit(1)
if not os.path.exists(val_path):
    print(f"Validation path {val_path} does not exist")
    raise SystemExit(1)
if not os.path.exists(test_path):
    print(f"Test path {test_path} does not exist")
    raise SystemExit(1)

# change directory in case some libs tries to deal with local/reletive paths
print(f"Current directory: {os.getcwd()}")
dir_content = os.listdir(os.getcwd())
print(f"Destination data path content: {dir_content}")
print(f"Switching to root destination data path '{root_dst_data_path}'")
os.chdir(root_dst_data_path)
print(f"New working directory: {os.getcwd()}")
      
# show target save directory
dir_content = os.listdir(root_dst_data_path)
print(f"Destination data path content: {dir_content}")
dir_content = os.listdir(save_dir)
print(f"Save directory content: {dir_content}")    
    

## data.yaml

YAML config for YOLO; note that because it's YAML file, it's not based on env-vars or globals


In [ ]:
#!/usr/bin/python

data_yaml_content = f"""
train: {train_path}
val: {val_path}
test: {test_path}

nc: 1  # number of classes
names: ['text']  # class names
"""

with open(data_yaml_file_path, 'w') as f:
    f.write(data_yaml_content)

# verify file now exists:
if not os.path.exists(data_yaml_file_path):
    print(f"Data yaml file {data_yaml_file_path} does not exist")

# Dump yaml content to verify, by reading it back
with open(data_yaml_file_path, 'r') as f:
    print(f.read())

## Training

```bash
yolo detect train data=data.yaml epochs=50 imgsz=640
```

Usage of Model size:

-   `yolov8n.pt` (Nano): The smallest model, optimized for speed and efficiency on resource-constrained devices. It has the least number of parameters and computational complexity, making it fast but less accurate.
-   `yolov8s.pt` (Small): A small model that offers a good balance between speed and accuracy. Suitable for scenarios where both performance and accuracy are important but resource usage needs to be moderate.
-   `yolov8m.pt` (Medium): A medium-sized model that improves accuracy over the small model but at the cost of additional computational resources and slower inference times.
-   `yolov8l.pt` (Large): A larger model with more parameters and higher computational requirements, offering higher accuracy but slower inference times.
-   `yolov8x.pt` (Extra Large): The largest model with the highest number of parameters and computational requirements. It provides the best accuracy but is the slowest in terms of inference speed.


In [ ]:
#!/usr/bin/python
import torch
from ultralytics import YOLO

########################### NOTE: ###########################
# This seems to only work on CoLab and not locally (maybe because
# I lack GPU, maybe because I was testing on Windows, etc)
#############################################################



# If `resume=False`:
# engine/trainer: task=detect, mode=train, model=yolov8s.pt,
#   data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml, epochs=50,
#   time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu,
#   workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True,
#   seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False,
#   amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4,
#   dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300,
#   half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,
#   augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,
#   save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,
#   show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False,
#   dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.937,
#   weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5,
#   dfl=1.5, pose=12.0, kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0,
#   translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0,
#   mixup=0.0, copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None,
#   tracker=botsort.yaml, save_dir=runs/detect/train
# If `resume=True`:
# engine/trainer: task=detect, mode=train, model=yolov8s.pt,
#   data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml,
#   epochs=500, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1,
#   cache=disk, device=cpu, workers=8, project=YOLOv8, name=yolov8s, exist_ok=False,
#   pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False,
#   rect=False, cos_lr=False, close_mosaic=10, resume=yolov8s.pt, amp=True, fraction=1.0,
#   profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=False,
#   val=True, split=val, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300,
#   half=True, dnn=False, plots=False, source=ultralytics/assets/, vid_stride=1, stream_buffer=False,
#   visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,
#   show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False,
#   show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript,
#   keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4,
#   nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.001, warmup_epochs=3.0,
#   warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, kobj=1.0,
#   label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1,
#   scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, mosaic=1.0, mixup=0.0,
#   copy_paste=0.0, auto_augment=randaugment, erasing=0.4, crop_fraction=1.0, cfg=None, tracker=botsort.yaml,
#   image_weights=False, hide_labels=False, hide_conf=False, line_thickness=3, fl_gamma=0.0, v5loader=True,
#   save_dir=YOLOv8/yolov8s



# NOTE: YOLO will set default value for 'save_dir=runs/detect/train' (if resume=False) and/or 'save_dir=YOLOv8/yolov8s' (if resume=True)
# print CURRENT directory:
# on CoLab, this will print '/content', as in '/content/YOLOv8/yolov8s/weights'
print(f"Current directory: {os.getcwd()}")

# Load the YOLO model
if os.path.exists('last.pt'):
    found_last = True
    model = YOLO('last.pt')
else:
    found_last = False
    # Note that YOLO will default download to `/content/yolov8s.pt` (on CoLab) because current directory is `/content`
    model = YOLO('yolov8s.pt')  # see MD coment above for other versions


# dump data.yaml content
with open(data_yaml_file_path, 'r') as f:
    print(f.read())


# if we're NOT running YOLO v8 (or higher), then we need to panic/exit

# GPU Count:
# ValueError: Invalid CUDA 'device=0,1,2,3' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.
# torch.cuda.is_available(): True
# torch.cuda.device_count(): 1
# os.environ['CUDA_VISIBLE_DEVICES']: None
devices = 'cpu'
if torch.cuda.is_available():
    devices = '0'   # assume we have only 1 GPU (Free: T4 GPU)
    if torch.cuda.device_count() > 1:
        devices = '0,1,2,3'


# Train the model, assume we don't have any GPU available, because of that (slow training), save every epoch (save_period=1)
# See: https://docs.ultralytics.com/reference/engine/trainer/
# Sadly, even if you set 'save_period=1', if the entire CoLab env shuts down/crashes/killed (i.e. you're only limited to 
# 2.5hrs of FREE GPU time), if you do not set your `save_dir` to a mounted Google Drive, you will lose all your progress
# because that CoLab instance went away!  So, make sure to save to Google Drive, or some other persistent storage
# so that you can resume/recover from the last successful epoch...
# NOTE: If setup correctly, you should be setup with at least 1 GPU and should float around the RAM=6GB, GPU RAM=4GB
#   engine/trainer: task=detect, mode=train, model=yolov8s.pt, 
# data=/content/drive/MyDrive/projects/ML-manga-ocr-rust/data/text_detection/data.yaml,
# epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1,
# cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, 
# pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, 
# single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, 
# amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, 
# overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, 
# save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, 
# source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, 
# agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,
# save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 
# show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, 
# optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, 
# nms=False, lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, 
# warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, dfl=1.5, pose=12.0, 
# kobj=1.0, label_smoothing=0.0, nbs=64, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, 
# translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, bgr=0.0, 
# mosaic=1.0, mixup=0.0, copy_paste=0.0, auto_augment=randaugment, erasing=0.4, 
# crop_fraction=1.0, cfg=None, tracker=botsort.yaml, save_dir=runs/detect/train
model.train(data=data_yaml_file_path, epochs=50,
            imgsz=640, save_period=1, device=devices, save_dir=save_dir)